In [1]:
!which python

/home/charx/python-virtual-environments/stonks-env/bin/python


In [48]:
from stonks.data.test_imports import say_hello
from pandas_datareader import data


# vanilla markotwitz + l2 regularized
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions 


# black litterman model
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier


import yfinance as yf
import pandas as pd
import numpy as np
import time

In [3]:
tickers = [
    'AMGN', # AMGEN inc -> biotech
    'A', # Agilent -> biotech
    'TMO', # Thermofisher -> biotech
    'VRTX', # Vertex phrmaceuticals -> biotech
    'BMRN', # Biomarine pharmaceuticals -> biotech
    'PFE', # Pfizer -> biotech
    'MRK', # Merk -> biotech
    'TSM', # Taiwan semi-conductor -> chips
    'QCOM', # Qualcomm -> chips
    'XLNX', # Xilix -> chips
    'NVDA', # NVIDIA -> chips (GPUS)
    'AMD', # AMD -> CPUs (chips)
    'ASML', # ASML chips
    'MU', # Micorn chips (storage)
    'BYND', # Beyond Meat food
    'ADM', # Archer Daniels food,
    'TSN', # Tyson foods
    'EL', # cosmetics
    'TWTR', # twitter -> social media
    'FB', # facebook -> social media
    'GOOGL', # google -> tech/cloud
    'MSFT', # microsoft -> tech/cloud
    'DDOG', # Datadog -> cloud
    'ABNB', # AIRBNB -> travel/tech
    'TEAM', # attlasian -> tech/cloud
    'VMW', # VMWare -> tech/cloud
    'DIS', # Disney -> travel/entretaiment
    'NFLX', # Netflix -> tech/entretainement
    'MNST', # Monster -> vicios Gen Z
]

In [4]:
# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2008-01-01'
end_date = '2021-03-04'

In [5]:
curr_df = data.DataReader('TMO', 'yahoo', start_date, end_date)

In [6]:
price_data = []
for ticker in tickers:
    print(ticker)
    curr_df = data.DataReader(ticker, 'yahoo', start_date, end_date)
    time.sleep(0.3)
    price_data.append(curr_df['Adj Close'])
    

AMGN
A
TMO
VRTX
BMRN
PFE
MRK
TSM
QCOM
XLNX
NVDA
AMD
ASML
MU
BYND
ADM
TSN
EL
TWTR
FB
GOOGL
MSFT
DDOG
ABNB
TEAM
VMW
DIS
NFLX
MNST


In [7]:
# construct a df with the company values
df_stocks = pd.concat(price_data, axis = 1)
df_stocks.columns = tickers

In [19]:
def back_fill_df(df, verbose = True):
    df_cp = df.copy()
    
    tkrs = list(df.columns)
    
    for tiker in tkrs:
        print(f'Backfilling: {tiker}') if verbose else 'continue'
        #srs = df_cp[df_cp[tiker].isnull()][tiker]
        srs = df_cp[~df_cp[tiker].isnull()][tiker]

        # only do the backfill if the series requires it
        if srs.shape[0] > 0:
            bkfill_val = srs[0]
            
            print(f"Backfilldate {srs.index[0]}") if verbose else 'continue'
            #bkfill_val = df_cp[df_cp.index == last_nan + pd.DateOffset(1)][ticker][0]
            
            print(f"Backfilling with {bkfill_val}") if verbose else 'continue'
            df_cp[tiker] = df_cp[tiker].fillna(value = bkfill_val)
    
    return df_cp

In [20]:
df_stocks_bkfilled = back_fill_df(df_stocks, verbose = True)

Backfilling: AMGN
Backfilldate 2008-01-02 00:00:00
Backfilling with 36.87551498413086
Backfilling: A
Backfilldate 2008-01-02 00:00:00
Backfilling with 23.827491760253906
Backfilling: TMO
Backfilldate 2008-01-02 00:00:00
Backfilling with 53.54214859008789
Backfilling: VRTX
Backfilldate 2008-01-02 00:00:00
Backfilling with 23.149999618530273
Backfilling: BMRN
Backfilldate 2008-01-02 00:00:00
Backfilling with 36.63999938964844
Backfilling: PFE
Backfilldate 2008-01-02 00:00:00
Backfilling with 12.73065185546875
Backfilling: MRK
Backfilldate 2008-01-02 00:00:00
Backfilling with 35.628963470458984
Backfilling: TSM
Backfilldate 2008-01-02 00:00:00
Backfilling with 5.873227596282959
Backfilling: QCOM
Backfilldate 2008-01-02 00:00:00
Backfilling with 27.58709144592285
Backfilling: XLNX
Backfilldate 2008-01-02 00:00:00
Backfilling with 15.640898704528809
Backfilling: NVDA
Backfilldate 2008-01-02 00:00:00
Backfilling with 30.349815368652344
Backfilling: AMD
Backfilldate 2008-01-02 00:00:00
Backfi

In [10]:
# get mean and cover matrices
mu = mean_historical_return(df_stocks_bkfilled)
S = CovarianceShrinkage(df_stocks_bkfilled).ledoit_wolf()

# with l2 regularization to not have 0s in the weights

ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.5)
w = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.txt")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('AMGN', 0.03842), ('A', 0.0), ('TMO', 0.04795), ('VRTX', 0.03129), ('BMRN', 0.0), ('PFE', 0.0), ('MRK', 0.0), ('TSM', 0.09129), ('QCOM', 0.0), ('XLNX', 0.02304), ('NVDA', 0.0331), ('AMD', 0.00608), ('ASML', 0.0909), ('MU', 0.00725), ('BYND', 0.01082), ('ADM', 0.0), ('TSN', 0.03068), ('EL', 0.08941), ('TWTR', 0.0), ('FB', 0.05454), ('GOOGL', 0.01605), ('MSFT', 0.03967), ('DDOG', 0.0148), ('ABNB', 0.0), ('TEAM', 0.09595), ('VMW', 0.0), ('DIS', 0.03331), ('NFLX', 0.18158), ('MNST', 0.06386)])
Expected annual return: 24.6%
Annual volatility: 22.2%
Sharpe Ratio: 1.02


/home/charx/python-virtual-environments/stonks-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimization problem so additional objectives may not work as expected."


(0.24625658844415546, 0.22192505951453703, 1.0195179802546577)

# Black litterman

In [11]:
mcaps = {}
for t in tickers:
    stock = yf.Ticker(t)
    mcaps[t] = stock.info["marketCap"]


In [12]:
mcaps

{'AMGN': 131529105408,
 'A': 35387625472,
 'TMO': 175974268928,
 'VRTX': 55106318336,
 'BMRN': 14133646336,
 'PFE': 191814696960,
 'MRK': 185042288640,
 'TSM': 561996627968,
 'QCOM': 147396001792,
 'XLNX': 29420974080,
 'NVDA': 309045198848,
 'AMD': 95112847360,
 'ASML': 213559230464,
 'MU': 99483320320,
 'BYND': 8704644096,
 'ADM': 32159064064,
 'TSN': 26519736320,
 'EL': 106248495104,
 'TWTR': 53436342272,
 'FB': 752582197248,
 'GOOGL': 1417468313600,
 'MSFT': 1746778193920,
 'DDOG': 25193641984,
 'ABNB': 107427127296,
 'TEAM': 57066651648,
 'VMW': 60333654016,
 'DIS': 344881266688,
 'NFLX': 228706566144,
 'MNST': 45641596928}

In [13]:
# get the market prices of the sp500
sp500 = yf.download("SPY", period="max")["Adj Close"]

# declare our views of the market
viewdict = {
    "ABNB": 0.40,
    "DIS": 0.30,
    "TSM": 0.40,
    "NVDA": 0.40,
    "QCOM": 0.40,
    "TWTR": 0.40,
    "DDOG": 0.20,
    "EL": 0.3,
    "MSFT": 0.3,
    "AMD": 0.4,
    "ASML": 0.5,
}

# how confident we are on our views
confidences = [
    0.3,
    0.8,
    0.8,
    0.7,
    0.8,
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
    0.9,
]


S = CovarianceShrinkage(df_stocks_bkfilled).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(sp500)

# get market priors
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)

# blmodel
bl = BlackLittermanModel(S, pi = market_prior, absolute_views = viewdict, risk_aversion = delta, omega="idzorek", view_confidences=confidences)

ret_bl = bl.bl_returns() # mean matrix for black litterman
S_bl = bl.bl_cov() # co-variance matrix for black litterman


# then we allocate the portfolio using markowitz regularized
ef = EfficientFrontier(ret_bl, S_bl)
#ef.add_objective(objective_functions.L2_reg, gamma = 0.5)
ef.add_objective(objective_functions.L2_reg)

ef.max_sharpe()
weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

[*********************100%***********************]  1 of 1 completed
Expected annual return: 31.3%
Annual volatility: 21.5%
Sharpe Ratio: 1.36


/home/charx/python-virtual-environments/stonks-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimization problem so additional objectives may not work as expected."


(0.3129777631349649, 0.21508591267733868, 1.362142966445579)

In [14]:
capital = 4768.50

total = 0
for k, v in weights.items():
    total += capital * v
    print(f"{k} pct: {v} amnt: {capital * v}")


AMGN pct: 0.0238 amnt: 113.4903
A pct: 0.036 amnt: 171.666
TMO pct: 0.02522 amnt: 120.26156999999999
VRTX pct: 0.02689 amnt: 128.224965
BMRN pct: 0.02584 amnt: 123.21803999999999
PFE pct: 0.0199 amnt: 94.89315
MRK pct: 0.01978 amnt: 94.32092999999999
TSM pct: 0.05362 amnt: 255.68697
QCOM pct: 0.05296 amnt: 252.53976
XLNX pct: 0.04076 amnt: 194.36406
NVDA pct: 0.0458 amnt: 218.3973
AMD pct: 0.032 amnt: 152.592
ASML pct: 0.07255 amnt: 345.954675
MU pct: 0.05222 amnt: 249.01107000000002
BYND pct: 0.02314 amnt: 110.34309
ADM pct: 0.02921 amnt: 139.287885
TSN pct: 0.02355 amnt: 112.298175
EL pct: 0.03643 amnt: 173.716455
TWTR pct: 0.04594 amnt: 219.06489000000002
FB pct: 0.02654 amnt: 126.55599000000001
GOOGL pct: 0.03979 amnt: 189.73861499999998
MSFT pct: 0.03889 amnt: 185.446965
DDOG pct: 0.02824 amnt: 134.66244
ABNB pct: 0.03316 amnt: 158.12346000000002
TEAM pct: 0.01827 amnt: 87.120495
VMW pct: 0.03491 amnt: 166.468335
DIS pct: 0.03612 amnt: 172.23821999999998
NFLX pct: 0.02941 amnt: 14

In [15]:
capital = 598

total = 0
for k, v in weights.items():
    total += capital * v
    print(f"{k} pct: {v} amnt: {capital * v}")

AMGN pct: 0.0238 amnt: 14.2324
A pct: 0.036 amnt: 21.528
TMO pct: 0.02522 amnt: 15.08156
VRTX pct: 0.02689 amnt: 16.08022
BMRN pct: 0.02584 amnt: 15.452319999999999
PFE pct: 0.0199 amnt: 11.9002
MRK pct: 0.01978 amnt: 11.828439999999999
TSM pct: 0.05362 amnt: 32.06476
QCOM pct: 0.05296 amnt: 31.67008
XLNX pct: 0.04076 amnt: 24.37448
NVDA pct: 0.0458 amnt: 27.3884
AMD pct: 0.032 amnt: 19.136
ASML pct: 0.07255 amnt: 43.3849
MU pct: 0.05222 amnt: 31.22756
BYND pct: 0.02314 amnt: 13.837720000000001
ADM pct: 0.02921 amnt: 17.46758
TSN pct: 0.02355 amnt: 14.0829
EL pct: 0.03643 amnt: 21.78514
TWTR pct: 0.04594 amnt: 27.47212
FB pct: 0.02654 amnt: 15.87092
GOOGL pct: 0.03979 amnt: 23.79442
MSFT pct: 0.03889 amnt: 23.25622
DDOG pct: 0.02824 amnt: 16.887520000000002
ABNB pct: 0.03316 amnt: 19.82968
TEAM pct: 0.01827 amnt: 10.925460000000001
VMW pct: 0.03491 amnt: 20.876179999999998
DIS pct: 0.03612 amnt: 21.59976
NFLX pct: 0.02941 amnt: 17.58718
MNST pct: 0.02903 amnt: 17.35994


# VaR

In [ ]:
#Backfilldate 2019-09-19 00:00:00

In [61]:
df = df_stocks_bkfilled.copy()
df_var = df[df.index > '2019-09-19']

# get the % vals of the stocks
weights_lst = list(weights.values())
weights_array = np.array(weights_lst)

In [50]:
sh_wt

array([0.0238 , 0.036  , 0.02522, 0.02689, 0.02584, 0.0199 , 0.01978,
       0.05362, 0.05296, 0.04076, 0.0458 , 0.032  , 0.07255, 0.05222,
       0.02314, 0.02921, 0.02355, 0.03643, 0.04594, 0.02654, 0.03979,
       0.03889, 0.02824, 0.03316, 0.01827, 0.03491, 0.03612, 0.02941,
       0.02903])

In [68]:
df_var = df_var.pct_change().iloc[1:,:]

In [77]:
#PnL = (weights * returns.values).sum(axis=1)
# calculate the daily historic PnL of the portfolio
hist_daily_portfolio_PnL = (weights_array * df_var.values).sum(axis = 1)

In [84]:
historic_var = np.percentile(hist_daily_portfolio_PnL, 5, interpolation="lower")

capital_pandors = 598
capital_puerquits = 4768.50

print(f'The simple historical VaR pandors is {historic_var * capital_pandors}')
print(f'The simple historical VaR puerquits is {historic_var * capital_puerquits}')


The simple historical VaR pandors is -16.417649761513978
The simple historical VaR puerquits is -130.9156570029756


In [85]:
historic_var

-0.027454263815240763

# CVAR

In [91]:
# filter all values with loses greater than the historic var
cvar = hist_daily_portfolio_PnL[hist_daily_portfolio_PnL <= historic_var]
print(f'The cvar of the portfolio is: {cvar.mean()}')
print(f'The simple historical CVaR pandors is {cvar.mean() * capital_pandors}')
print(f'The simple historical VaR puerquits is {cvar.mean() * capital_puerquits}')

The cvar of the portfolio is: -0.04671681053335994
The simple historical CVaR pandors is -27.936652698949242
The simple historical VaR puerquits is -222.76911102832685
